<a href="https://colab.research.google.com/github/crosoriom/Senales-y-sistemas/blob/main/Parcial1_SyS_2024_2_CristianCamiloOsorioMesa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, entrada análoga de -3.3 a 3.3 [v], aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits, rango de la entrada análoga y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

\begin{equation}
    x(t) = 0.3 \cos\left(1000\pi t - \frac{\pi}{4}\right) + 0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t - \pi)
\end{equation}

Para una señal cuasiperiodica, se tiene que su periodo $T$ será de:

$$ T = \frac{2\pi}{\omega} \quad ; \quad f = \frac{1}{T} \quad ; \quad f = \frac{\omega}{2\pi} $$

\begin{align*}
T_1 &= \frac{1}{f_1} = \frac{1}{1000\pi/(2\pi)} = \frac{1}{500} = 0.002 \\
T_2 &= \frac{1}{f_2} = \frac{1}{2000\pi/(2\pi)} = \frac{1}{1000} = 0.001 \\
T_3 &= \frac{1}{f_3} = \frac{1}{11000\pi/(2\pi)} = \frac{1}{5500} = 0.00018181818...
\end{align*}

\begin{equation}
T = \text{mcm}(T_1, T_2, T_3) = 0.002 \\
f = \text{mcd}(f_1, f_2, f_3) = 500 \\
T = \frac{1}{f} = \frac{1}{500} = 0.002
\end{equation}


In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

#Consstantes de la función:
A1 = 0.3
A2 = 0.6
A3 = 0.1
w1 = 1000 * np.pi
w2 = 2000 * np.pi
w3 = 11000 * np.pi
phi1 = -np.pi/4
phi2 = 0
phi3 = -np.pi

#Periodo y frecuencia:
T1 = 2 * np.pi / w1
T2 = 2 * np.pi / w2
T3 = 2 * np.pi / w3
f1 = 1 / T1
f2 = 1 / T2
f3 = 1 / T3

f = 500
T = 1 / f
w = 2 * np.pi * f

#Por teorema de Nyquist Fs>= 2f, 5000 es 10 veces mayor a 500
Fs = 5000

#Función continua y analógica
t = sp.symbols('t')
x = A1 * sp.cos(w1 * t + phi1) + A2 * sp.sin(w2 * t + phi2) + A3 * sp.cos(w3 * t + phi3)
sp.plot(x, (t, -2 * T, 2 * T), title = "x(t) análogica y continua")

#Función analógica discretizada
plt.figure(figsize=(20, 4))
for i in range(1, 6):
    tn = np.arange(-2 * T, 2 * T, 1/(i * Fs))
    xn = A1 * np.cos(w1 * tn + phi1) + A2 * np.sin(w2 * tn + phi2) + A3 * np.cos(w3 * tn + phi3)

    plt.subplot(1,5,i)
    plt.plot(tn, xn)
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud')
    plt.title(f'Fs = {(i * Fs)}')
    plt.grid(True)
plt.show()

Aunque el teorema de Nyquist se cumple para frecuencias mayores o iguales a $1kHz$, con la frecuencia de muestreo del ADC aparece cierto nivel de aliasing, puede que esto sea deseable ya que el ruido puede verse filtrado. Si esto no se desea, y lo que se quiere es ver la señal tal cual se recibe de forma analógica, se debe usar una mayor frecuencia. Se sugiere mayor a $15kHz$.

In [ ]:
#Se usará la frecuencia de muestreo de 25kHz.
#Función para aplicar cero y pendiente
def GainAndOffset(x, y_min, y_max):
    x_min = min(x)
    x_max = max(x)
    gain = (y_max - y_min) / (x_max - x_min)
    offset = y_min - gain * x_min
    y = gain * x + offset
    return y

#Función para cuantizar la amplitud
def Quantizer(y, v) :
    N = v.shape[0]
    d = cdist(y.reshape(-1,1),v.reshape(-1,1))
    n = np.argmin(d)
    return v[n]

#Primero se atenúa la señal reduciendo su ganancia y calculando un offset
x_max = 3.3
x_min = -3.3
xn_r = GainAndOffset(xn, x_min, x_max)

#Luego se cuantiza la amplitud de la señal para volverla completamente digital.
bits = 5
v = np.linspace(x_min, x_max, 2**bits)
xn_c = np.zeros(xn_r.shape)
for i in range(xn_r.shape[0]):
    xn_c[i] = Quantizer(xn_r[i], v)

plt.figure(figsize=(10, 5))
plt.subplot(1,2,1)
plt.plot(tn, xn_r)
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señal analógica discretizada y regularizada')
plt.grid(True)

plt.subplot(1,2,2)
#for value in v:
#    plt.plot(tn, value * np.ones(len(tn)))
plt.plot(tn, xn_c)
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señal digitalizada')
plt.grid(True)
plt.show()


Puede observarse que tras la digitalización completa de la señal, apenas se notan ligeros cambios en la señal. En la discretización se observa que los cambios en las pendientes son más bruscos, apareciendo picos y no curvas en los máximos y mínimos de la señal. En la cuantización hay cambios en los mínimos, algunos puntos mínimos se notan ligeramente desplazados.

Se usó la IA para escribir la demostración de la periodicidad de la función en TeX. Y resolver algunos problemas con los paquetes usados en el código.
**Olvidé los prompts**

# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

Para que un sistema sea lineal e invariante en el tiempo(SLIT) se deben cumplir 2 condiciones:

1. $$T\{a_{1}x_{1}\ + a_{2}x_{2}\} = a_{1}y_{1} + a_{2}y_{2}$$
2. $$T\{x[n-k]\} = y[n-k]$$


In [ ]:
y_ax = my_system.predict((signal_u) + my_system.predict(5 * signal_u)) #Se suman señales para comprobar superposición
signal_k = np.heaviside(t - 0.01,1) #Se traslada la entrada para ver si traslada la salida
y_k = my_system.predict(signal_k)

fig, axs = plt.subplots(2,1)
axs[0].stem(t, y_ax, label='Propiedad de superposición')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_k, label='Propiedad de traslación')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

Comprobadas las propiedades, se asume que el sistema es un SLIT. Ahora se calcula su respuesta al impulso $h(t)$.

In [ ]:
h = np.gradient(signal_u, t) #señal impulso
y_h = my_system.predict(h)

fig, axs = plt.subplots(2,1)
axs[0].stem(t, h, label='función impulso')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_h, label='Respuesta impulso')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

Finalmente se hace la convolución con $x(t)$ para hallar la respuesta del sistema.

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
x = np.sin(100 * np.pi * t) + np.sin(600 * np.pi * t)
t_n = np.arange(-0.01, 0.049, 1/fs)  # Nuevo vector de tiempo
y = np.convolve(x.reshape(-1),h.reshape(-1))

plt.stem(t_n, y, label='Respuesta del sistema')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.legend()
plt.grid()
plt

Para este punto pedí asesoría en Gemini para solucionar algunas dudas que tuve con numpy. Calculé un nuevo vector de tiempo para la salida del sistema debido a que la convolución arroja un vector de la suma de las longitudes de las entradas.

**Conversación con Gemini:**

np puede hacer derivadas?

Gemini
Sí, NumPy puede hacer derivadas, pero de forma numérica, no simbólica. Esto significa que calcula la derivada en puntos específicos en lugar de encontrar una expresión general para la derivada.

Aquí te muestro cómo puedes usar NumPy para calcular la derivada de una función:

Pasos:

Importar NumPy:

import numpy as np
Use code with caution
Definir la función:

def f(x):
  return x**2  # Ejemplo: función x^2
Use code with caution
Crear un array de puntos donde se evaluará la derivada:

x = np.linspace(-5, 5, 100)  # 100 puntos entre -5 y 5
Use code with caution
Calcular la derivada usando np.gradient:

derivada = np.gradient(f(x), x)
Use code with caution
np.gradient calcula la diferencia central de la función f(x) con respecto a x, lo cual es una aproximación de la derivada.

Razonamiento:

NumPy no tiene funciones para derivación simbólica como las que podrías encontrar en SymPy. Sin embargo, np.gradient ofrece una forma eficiente de calcular la derivada numérica, que es suficiente para muchos propósitos en ciencia de datos y aprendizaje automático.

Ejemplo completo:


import numpy as np

def f(x):
  return x**2

x = np.linspace(-5, 5, 100)
derivada = np.gradient(f(x), x)

# Para visualizar la derivada:
import matplotlib.pyplot as plt
plt.plot(x, derivada)
plt.xlabel('x')
plt.ylabel('Derivada de f(x)')
plt.title('Derivada numérica usando NumPy')
plt.grid(True)
plt.show()
Use code with caution
To see the output, run the code.

Espero que esto te ayude. Avísame si tienes alguna otra pregunta.

Califica esta respuesta

Cristian Camilo Osorio Mesa
y np contiene la función impulso?

Gemini
NumPy en sí mismo no tiene una función específica llamada "impulso" o "delta de Dirac" que represente directamente la función impulso unitario ideal. Sin embargo, puedes representarla y trabajar con ella de manera aproximada usando arrays de NumPy.

Aquí te muestro dos enfoques comunes:

1. Aproximación con un pulso rectangular:

Puedes crear un array que tenga un valor alto en un solo punto (representando el impulso) y cero en el resto.


import numpy as np

# Duración del impulso (ajusta según tu necesidad)
duracion = 10

# Crear un array de ceros
impulso = np.zeros(duracion)

# Establecer el punto central como 1 (o el valor deseado del impulso)
impulso[duracion // 2] = 1
Use code with caution
Razonamiento:

Este método representa el impulso como un pulso rectangular estrecho. Mientras más corto sea el pulso, mejor se aproximará a la función delta de Dirac ideal.

2. Usando una función gaussiana:

Otra opción es usar una función gaussiana con una desviación estándar muy pequeña, que también se aproxima a la forma de un impulso.


import numpy as np

# Desviación estándar (ajusta para controlar la anchura del impulso)
desviacion_estandar = 0.1  

# Crear un array de tiempo
tiempo = np.linspace(-5, 5, 100)

# Calcular la función gaussiana
impulso = np.exp(-tiempo**2 / (2 * desviacion_estandar**2))  

# Normalizar para que el área bajo la curva sea 1 (opcional)
impulso /= np.trapz(impulso, tiempo)
Use code with caution
Razonamiento:

La función gaussiana es continua y suave, lo cual puede ser útil en algunos casos. Ajustando la desviación estándar, puedes controlar la anchura del impulso.

Importante:

Recuerda que estas son solo aproximaciones. La función delta de Dirac ideal es un concepto matemático con propiedades especiales que no se pueden representar perfectamente con un array numérico.

Espero que esto te ayude a trabajar con la función impulso en NumPy.

Califica esta respuesta

Cristian Camilo Osorio Mesa
Podrías ayudarme con este error por favor? ValueError Traceback (most recent call last) in <cell line: 4>() 2 y = np.convolve(x.reshape(-1),h.reshape(-1)) 3 ----> 4 plt.stem(t, y, label='Respuesta del sistema') 5 plt.xlabel('Tiempo (s)') 6 plt.ylabel('Amplitud')

6 frames /usr/local/lib/python3.10/dist-packages/numpy/ma/core.py in setitem(self, indx, value) 3385 if _mask is nomask: 3386 # Set the data, then the mask -> 3387 _data[indx] = dval 3388 if mval is not nomask: 3389 _mask = self._mask = make_mask_none(self.shape, _dtype)

ValueError: could not broadcast input array from shape (59,) into shape (30,)

Gemini
El error ValueError: could not broadcast input array from shape (59,) into shape (30,) surge porque estás intentando asignar un array de forma (59,) a un array de forma (30,). Esto suele ocurrir cuando intentas graficar dos arrays de diferentes tamaños con plt.stem u otras funciones de Matplotlib.

En tu caso, parece que el resultado de la convolución (y) tiene una longitud de 59, mientras que tu array de tiempo (t) tiene una longitud de 30. Para solucionar este problema, necesitas asegurarte de que ambos arrays tengan la misma longitud.

Aquí te muestro cómo puedes solucionarlo:

Razonamiento:

La convolución de dos señales produce una señal de salida con una longitud mayor que la de las señales de entrada. Por lo tanto, el array y resultante de np.convolve será más largo que t. Necesitas crear un nuevo array de tiempo que sea lo suficientemente largo para contener la señal de salida.

Pasos:

Calcula la longitud del array de salida:

len_y = len(x) + len(h) - 1
Use code with caution
(La longitud de la convolución es la suma de las longitudes de las entradas menos 1)

Crea un nuevo array de tiempo (t_conv) con la longitud correcta:

t_conv = np.linspace(t[0], t[-1] + (len_y - len(t)) * (t[1] - t[0]), len_y)
Use code with caution
Esto crea un array de tiempo que comienza en el mismo punto que t y termina en un punto que se extiende lo suficiente para abarcar la señal de salida de la convolución.

Grafica usando el nuevo array de tiempo:

plt.stem(t_conv, y, label='Respuesta del sistema')
Use code with caution
Explicación:

El nuevo array de tiempo t_conv se crea usando np.linspace para garantizar que tenga la misma longitud que el array y resultante de la convolución.
Se calcula el espaciado entre los puntos de tiempo en el array t original (t[1] - t[0]) y se utiliza para extender el array t_conv a la longitud adecuada.
Ejemplo:


import numpy as np
import matplotlib.pyplot as plt

# ... (tu código anterior para definir x, h, t) ...

y = np.convolve(x.reshape(-1),h.reshape(-1))
len_y = len(x) + len(h) - 1
t_conv = np.linspace(t[0], t[-1] + (len_y - len(t)) * (t[1] - t[0]), len_y)

plt.stem(t_conv, y, label='Respuesta del sistema')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.legend()
plt.show()
Use code with caution
To see the output, run the code.

Espero que esto te ayude a resolver el error y a visualizar correctamente la respuesta del sistema.

Califica esta respuesta

Cristian Camilo Osorio Mesa
por qué la convolución me genera un vector más largo?

Gemini
La convolución produce un vector más largo que los vectores de entrada debido a la forma en que se realiza el proceso matemático. Imagina la convolución como un proceso de "deslizamiento y multiplicación" entre las dos señales de entrada.

Aquí te explico el razonamiento:

Deslizamiento: Una señal (llamada kernel o filtro) se desliza sobre la otra señal (la señal de entrada).
Multiplicación: En cada posición del deslizamiento, se multiplican los valores correspondientes de ambas señales.
Suma: Se suman los resultados de las multiplicaciones para obtener un único valor de salida para esa posición.
La clave está en los puntos de inicio y fin:

Inicio: Cuando el kernel comienza a deslizarse sobre la señal de entrada, solo una parte del kernel se superpone con la señal. Esto produce valores de salida iniciales.
Medio: A medida que el kernel se desliza más hacia el centro, se superpone completamente con la señal de entrada. Esto produce la mayor parte de los valores de salida.
Fin: Cuando el kernel llega al final de la señal de entrada, solo una parte del kernel se superpone. Esto produce valores de salida finales.
Como resultado, la señal de salida tiene una longitud que es la suma de las longitudes de las señales de entrada menos 1.

Piensa en ello como la combinación de todas las posibles superposiciones entre las dos señales.

Ejemplo:

Imagina que tienes una señal de entrada de longitud 5 y un kernel de longitud 3.

En la primera posición del deslizamiento, solo el primer elemento del kernel se superpone con la señal de entrada.
En la segunda posición, los dos primeros elementos del kernel se superponen.
Y así sucesivamente, hasta que en la séptima posición, solo el último elemento del kernel se superpone con el último elemento de la señal de entrada.
Por lo tanto, la señal de salida tendrá una longitud de 7 (5 + 3 - 1).

En resumen:

La convolución produce un vector más largo porque tiene en cuenta todas las posibles superposiciones entre las dos señales de entrada, lo que genera valores de salida adicionales al principio y al final. Esto es fundamental para capturar la influencia completa del kernel en la señal de entrada.

Espero que esta explicación te ayude a entender por qué la convolución genera un vector más largo.

Califica esta respuesta

Cristian Camilo Osorio Mesa
Entonces aclarame algo, las funciones x yh están definidas sobre t, al hacer la convolución, me sale una función nueva, esta función a que valores de t correspondería?

Gemini
Buena pregunta. Cuando convolucionas dos funciones x y h que están definidas sobre un rango de tiempo t, la nueva función resultante, y, también está definida sobre un rango de tiempo, pero este rango se modifica debido al proceso de convolución.

Explicación:

Rango original de t: x(t) y h(t) están definidas para un rango específico de valores de t, por ejemplo, desde t_inicio hasta t_fin.
Convolución: La convolución y(t) = x(t) * h(t) implica integrar el producto de x y h para todos los posibles valores de desplazamiento entre ellas.
Nuevo rango de tiempo: El nuevo rango de tiempo para y(t) se extiende debido a la "cola" que se agrega por la superposición de las señales durante la convolución.
En términos más precisos:

Si x(t) está definida para t en el rango [t_inicio_x, t_fin_x] y h(t) está definida para t en el rango [t_inicio_h, t_fin_h], entonces y(t) estará definida para t en el rango [t_inicio_x + t_inicio_h, t_fin_x + t_fin_h].
Ejemplo simple:

Imagina que x(t) y h(t) son pulsos rectangulares de duración 1 segundo, centrados en t=0.

x(t) está definida para t en el rango [-0.5, 0.5]
h(t) está definida para t en el rango [-0.5, 0.5]
La convolución y(t) será un pulso triangular definido para t en el rango [-1, 1] (observa que se ha extendido).
En tu caso específico:

Si tus funciones x y h están definidas sobre el array de tiempo t, entonces el resultado de la convolución y no estará directamente definido sobre el mismo array t. Necesitas crear un nuevo array de tiempo, t_conv, como se explicó en la respuesta anterior, para representar correctamente el rango de tiempo de la señal de salida y.

En resumen:

La convolución extiende el rango de tiempo de la señal resultante. Debes ajustar el array de tiempo para que coincida con este nuevo rango y puedas visualizar la señal de salida correctamente.

Espero que esta aclaración te sea útil.

Califica esta respuesta
